In [1]:
# start
%matplotlib widget
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
from functools import partial
import matplotlib.pyplot as plt
import src.FeatureExplorer.SampleDataController as datacontrol
import src.FeatureExplorer.FeatureViewer as vc

import src.ModelEvaluator.GridSearch as gs
import src.ModelEvaluator.ModelResultsController as vr
import src.ModelEvaluator.ModelResultsViewer as evc

import  src.ModelEvaluator.ManualTestViewer as mtv

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


df = datacontrol.SampleData('datasets/RAVDESS/metadata/RAVDESS.csv')
vc.view_feature_controls(df)
gridsearch = gs.GridCV(df)
results = gridsearch.perform_gridsearch()

1317
auto-splitting: mels = 128, mfcc = 40, split 60/40 training/testing
GridSearchCV took 9.06 seconds for 28 candidate parameter settings.


In [52]:
# Visualizers
import librosa.display
import matplotlib.pyplot as plt
import src.FeatureExplorer.FeatureExtractors as fe
from matplotlib.colors import Normalize
from sklearn.metrics import ConfusionMatrixDisplay,RocCurveDisplay,PrecisionRecallDisplay,DetCurveDisplay
import numpy as np

from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go

plt.ioff()

class SpectroVisualizer:

    def __init__(self):
        self.fig = plt.figure(figsize=(5,7), layout="constrained")
        
        
    def get_record_viz(self,af,num_mels_filters=128, mel_fmax=8000, num_mfcc_filters=40,):
        
        self.fig.clf()

        self.axs = self.fig.subplot_mosaic(
            """
            AAAA
            BBBB
            CCCC
            """)

        mel = librosa.display.specshow(librosa.power_to_db(af.get_melspectrogram(num_mels_filters, mel_fmax), ref=np.mean),
                                                        y_axis='mel',fmax=mel_fmax, norm=Normalize(vmin=-20,vmax=20), ax=self.axs["A"])
        
        self.axs["A"].set(title='mel spectrogram')
        self.axs["A"].tick_params(axis='y',labelsize=7)
        mcb = self.fig.colorbar(mel,format='%+2.0f dB')
        mcb.ax.tick_params(axis='y',labelsize=7)

        mfcc = librosa.display.specshow(af.get_mfcc(num_mfcc_filters),norm=Normalize(vmin=-30,vmax=30), ax=self.axs['B'])
        self.axs['B'].set(title='mfcc')
        self.axs['B'].tick_params(axis='y',labelsize=7)
        mfc = self.fig.colorbar(mfcc)
        mfc.ax.tick_params(axis='y',labelsize=7)

        wav = librosa.display.waveshow(af.get_waveform(), sr=af.get_sample_rate(),axis='time', ax=self.axs['C'])
        self.axs['C'].set(title='wave')
        self.axs['C'].tick_params(axis='y',labelsize=7)

        self.fig.canvas.header_visible = False
        
        return self.fig
    
class RecordMetaViewer:

    def __init__(self):
        

        self.fig = plt.figure(figsize=(2,1))

    def show_record_metadata(self,emotion, sex, id):
        
        self.fig.clf()
        axs = self.fig.subplots()
        self.fig.set_size_inches(2.2, 1)
        cols = ['emotion','actor_sex','actor_id']
        cells = [emotion, sex, str(id)]

        self.fig.canvas.header_visible = False
        self.fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        self.fig.tight_layout()
        table = axs.table( colLabels=cols,cellText=[cells],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1.5,1.5)
        #plt.close()
        return self.fig
    

class ViewGridSearchResults:
    
    def __init__(self):
        self.fig  = plt.figure(figsize=(4,3))

    def print_records_to_table(self, records):
        
        self.fig.clf()
        
        cols = records[0]
        cell_text = records[1]

        ax = self.fig.subplots()
        self.fig.suptitle("Top Grid Search Results")
        self.fig.set_size_inches(4, 3)
        self.fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')
        self.fig.tight_layout()
        self.fig.canvas.header_visible = False
        
        col_widths = [0.15,0.16,0.17,0.16,0.18,0.18]

        table = ax.table(cellText=cell_text,colLabels=cols,loc='center',colWidths=col_widths)
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1.0,1.5)
       
        return self.fig


class ViewPrecisionRecall:
    

    def __init__(self):

        self.fig = plt.figure(figsize=(4,3))
       
        
    def show_precision_recall(self,train_model,features_train,features_test,labels_train,labels_test,test_model,test_mode):
        self.fig.clf()
        
        ax = self.fig.subplots()
        self.fig.set_size_inches(4,3)
        self.fig.canvas.header_visible = False
        ax.set_title("Precision Recall Curve (PRC)")
        PrecisionRecallDisplay.from_estimator(train_model, features_train, labels_train,ax=ax,name='train',plot_chance_level=True) 
        
        if test_mode == 1:
            pred = test_model.decision_function(features_test)
            PrecisionRecallDisplay.from_predictions(labels_test,pred,ax=ax,name="test",plot_chance_level=True)
        ax.set_aspect('auto')
        #ax.set_box_aspect(0.7)
        ax.set_ylabel('Precision')
        ax.set_xlabel('Recall')
        ax.xaxis.set_label_coords(0.6, -.15)
        
        ax.legend(bbox_to_anchor=(-0.1, -0.13), loc='upper left', borderaxespad=0.1)

        self.fig.tight_layout()
        return self.fig



class ViewConfusionMatrix:
    
    def __init__(self):
        self.train_fig = plt.figure(figsize=(4,3))
        self.test_fig = plt.figure(figsize=(4,3))

    def show_confusion_matrix_train(self,model,features_train,labels_train):
        
        self.train_fig.clf()
        self.train_fig.set_size_inches(3,3)
        axs = self.train_fig.subplots()
        axs.set_title("Confusion Matrix Train",fontsize=12)
        
        ConfusionMatrixDisplay.from_estimator(model, features_train, labels_train,ax=axs,colorbar=False)
        self.train_fig.tight_layout()
        return self.train_fig
    
    def show_confusion_matrix_test(self,test_mode, labels_test,predictions):
        self.test_fig.clf()
        self.test_fig.set_size_inches(3,3)
        axs = self.test_fig.subplots()
        axs.set_title("Confusion Matrix Test",fontsize=12)
        if test_mode == 1:
            ConfusionMatrixDisplay.from_predictions(labels_test,predictions,ax=axs,colorbar=False,cmap="magma")
        self.test_fig.tight_layout()
        return self.test_fig


class ViewModelMetrics:
    
    def __init__(self):

        self.fig = plt.figure(figsize=(2,1.25))

    def show_train_metrics(self,train_record):
        
        self.fig.clf()

        cols = train_record[0]
        cell_text = train_record[1]

        self.fig.set_size_inches(2,1.25)
        rows = ["Train Results"]
        axs = self.fig.subplots()
        self.fig.suptitle("Model Performance")
        self.fig.canvas.header_visible = False
        self.fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        self.fig.tight_layout()
        table = axs.table( colLabels=cols,rowLabels=rows,cellText=[cell_text],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1.5,1.5)
        
        return self.fig

    def show_test_metrics(self,train_record, test_record):
      
        self.fig.clf()
        self.fig.set_size_inches(2,1.5)
        cols = train_record[0]
        train_row = train_record[1]
        test_row = test_record[1]
        
        rows = ["Training","Testing"]
        axs = self.fig.subplots()
        self.fig.suptitle("Model Performance")
        self.fig.canvas.header_visible = False
        self.fig.patch.set_visible(False)
        axs.axis('off')
        axs.axis('tight')
        self.fig.tight_layout()
        table = axs.table( colLabels=cols,rowLabels=rows,cellText=[train_row, test_row],loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(8)
        table.scale(1.5,1.5)
        
        return self.fig

class ViewROC:

    def __init__(self):
        self.fig = plt.figure(figsize=(6,3))


    def show_ROC(self,train_model,features_train,features_test,labels_train,labels_test,test_model,test_mode):

        self.fig.clf()
        self.fig.set_size_inches(6,3)
        ax = self.fig.subplots()
        ax.set_title("ROC Curve")
        RocCurveDisplay.from_estimator(train_model, features_train, labels_train,ax=ax,name="train") 
        
        if test_mode == 1:
            pred = test_model.decision_function(features_test)
            RocCurveDisplay.from_predictions(labels_test,pred,ax=ax,name="test")
        
        self.fig.canvas.header_visible = False
        ax.set_aspect('auto')
        ax.set_ylabel('True Positive Rate')
        ax.set_xlabel('False Positive Rate')
        #self.fig.tight_layout()
        return self.fig

  


class ViewDET:
    
    def __init__(self):
        self.fig = plt.figure(figsize=(3,3), layout="constrained")

    def show_DET(self,train_model,features_train,features_test,labels_train,labels_test,test_model,test_mode):
        self.fig.clf()
        self.fig.set_size_inches(3,3)
        ax = self.fig.subplots()
        ax.set_title("DET Curve")
        self.fig.canvas.header_visible = False
        DetCurveDisplay.from_estimator(train_model, features_train, labels_train,ax=ax,name='train')
        if test_mode == 1:
            pred = test_model.decision_function(features_test)
            DetCurveDisplay.from_predictions(labels_test,pred,ax=ax,name='test')
      
        ax.set_ylabel('False Negative Rate')
        ax.set_xlabel('False Positive Rate')
        self.fig.tight_layout()
        return self.fig

In [53]:
# Controller
from sklearn.metrics import ConfusionMatrixDisplay,RocCurveDisplay,PrecisionRecallDisplay,DetCurveDisplay
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score,precision_score,accuracy_score,d2_absolute_error_score
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import src.util.RecordVisualizer as rv



from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score,precision_score,accuracy_score,d2_absolute_error_score
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt


class ResultsViewer:

    def __init__(self, df, sample_data):
        
        self.df = self.arrange_columns(df)
        
        self.model = SVC()
        self.current_record_idx = 0

  

        self.features_train = sample_data.features_train
        self.labels_train = sample_data.labels_train

        # Visualizers
        ''' 
        self.vgr = rv.ViewGridSearchResults()
        self.prc = rv.ViewPrecisionRecall()
        self.vcm = rv.ViewConfusionMatrix()
        self.vmm = rv.ViewModelMetrics()
        self.vroc = rv.ViewROC()
        self.vdet = rv.ViewDET()
        '''

        self.vgr = ViewGridSearchResults()
        self.prc = ViewPrecisionRecall()
        self.vcm = ViewConfusionMatrix()
        self.vmm = ViewModelMetrics()
        self.vroc = ViewROC()
        self.vdet = ViewDET()




        # Testing only
        self.features_test = sample_data.features_test
        self.labels_test = sample_data.labels_test
        self.test_model = None
        self.test_mode = 0

        self.predictions = []


    def arrange_columns(self, in_df):
        df = in_df.copy()
        df['rank']=list(range(1,len(df)+1))
        df.drop(columns=['rank_test_score'],inplace=True)
        df.columns=['L2Mult','weight','gamma','test_recall','train_recall', 'test_stdev','rank']
        df = df[['rank','L2Mult','weight','gamma','test_recall','train_recall', 'test_stdev']]
        return df
    

    def get_top_records(self, num_records=10):
        return self.df.iloc[0:num_records]

    def format_top_records_table(self, recs,num_records=10):

        cols = ['rank', 'weight','gamma','L2Mult','test recall', 'train recall']

        cell_text = []
        for i in range(num_records):
            
            weight = 'bal'
            if recs['weight'].iloc[i] == None:
                weight ="unbal"
            
            cell_text.append([recs['rank'].iloc[i], 
                              weight,
                              recs['gamma'].iloc[i],
                              round(recs['L2Mult'].iloc[i],7),
                              round(recs['test_recall'].iloc[i],2),
                              round(recs['train_recall'].iloc[i],2)])
        return cols, cell_text

    def print_records_to_table(self):
        return self.vgr.print_records_to_table(self.format_top_records_table(self.get_top_records()))

  ####################################################################################
    
    def select_record(self,idx):
        plt.close()
        self.current_record_idx = idx
        self.fit_model_with_record(idx)

    
    def apply_record_to_model(self, rec_index=0):
        
        return SVC(C=self.df['L2Mult'].iloc[rec_index],
                   kernel='rbf',
                   gamma=self.df['gamma'].iloc[rec_index],
                   class_weight=self.df['weight'].iloc[rec_index]
                   )
     
           
    def fit_model_with_record(self, rec_index=0):    
        self.model = self.apply_record_to_model(rec_index=rec_index)
        self.model = self.model.fit(self.features_train,self.labels_train)
        return self.model
       

    def set_testing_model(self):
        self.test_model = self.model
        self.test_mode = 1
        self.predictions = self.test_model.predict(self.features_test)

    def get_model_used_for_test(self):
        return self.test_model


    def show_confusion_matrix_train(self):
        return self.vcm.show_confusion_matrix_train(self.model,
                                                    self.features_train,
                                                    self.labels_train)
   

    def show_confusion_matrix_test(self):
        return self.vcm.show_confusion_matrix_test(self.test_mode,
                                                   self.labels_test,
                                                   self.predictions)
    
    def show_ROC(self):
        return self.vroc.show_ROC(self.model,
                                self.features_train,
                                self.features_test,
                                self.labels_train,
                                self.labels_test,
                                self.test_model,
                                self.test_mode)
    
        
      
    def show_DET(self):
        return self.vdet.show_DET(self.model,
                                self.features_train,
                                self.features_test,
                                self.labels_train,
                                self.labels_test,
                                self.test_model,
                                self.test_mode)


    def show_precision_recall(self):
        return self.prc.show_precision_recall(self.model,
                                              self.features_train,
                                              self.features_test,
                                              self.labels_train,
                                              self.labels_test,
                                              self.test_model,
                                              self.test_mode)


    def get_train_metrics(self):
      
        scoring = ['recall','precision','accuracy']
        scores = cross_validate(self.model, self.features_train, self.labels_train,scoring=scoring)
        
        cols = ['recall','precision','accuracy']
        cells = []
        cells.append(round(scores['test_recall'].mean(),2))
        cells.append(round(scores['test_precision'].mean(),2))
        cells.append(round(scores['test_accuracy'].mean(),2))
        #cells.append(round(scores['test_d2_absolute_error_score'].mean(),2))

        return cols, cells


    def get_test_metrics(self):
        pred = self.test_model.decision_function(self.features_test)
        cols = ['recall','recall_micro','precision']
        cells = []
        cells.append(round(recall_score(self.labels_test, self.predictions),3))
        cells.append(round(precision_score(self.labels_test, self.predictions),3))
        cells.append(round(accuracy_score(self.labels_test, self.predictions),3))
        #cells.append(round(d2_absolute_error_score(self.labels_test,self.predictions),3))
        
        return cols, cells
    
    def show_train_metrics(self):
        return self.vmm.show_train_metrics(self.get_train_metrics())


    def show_test_metrics(self):
        return self.vmm.show_test_metrics(self.get_train_metrics(),self.get_test_metrics())

In [54]:
# Viewer
def eval_controller(tr):


    # layouts

    stretch_records = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='30%')

    stretch_roc = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    align_content='stretch',
                    justify_content='center',
                    width='30%')

    stretch_stats = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='35%',
                    margin='0px 0px 0px 10px')

    center_align = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%')


    stat_box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    align_content='center',
                    width='90%',
                    justify_content='center',
                    margin='0px 0px 0px 0px')

    top_row_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_content='flex-start',
                    width='100%',
                    justify_content='flex-start')
    
    bottom_row_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_content='flex-start',
                    width='100%',
                    justify_content='flex-start')

    prec_test_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_content='flex-start',
                    align_items='stretch',
                    width='30%',
                    justify_content='flex-start',
                    margin='10px 0px 0px 0px')

    cm_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_content='stretch',
                    align_items='stretch',
                    width='35%',
                    justify_content='flex-start',
                    margin='10px 10px 0px 0px')

    det_layout = widgets.Layout(display='flex',
                   
                    flex_flow='row',
                    align_content='center',
                    align_items='center',
                    width='30%',
                    justify_content='center',
                    margin='10px 0px 0px 0px')

    oos_test_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='20%',
                    justify_content='center')




    record_out = widgets.Output(layout=stretch_records)
    roc_out = widgets.Output(layout=stretch_roc)
    stats_out =  widgets.Output(layout=stretch_stats)
    det_out = widgets.Output(layout=det_layout)

    cm_out = widgets.Output(layout=center_align)
    cm2_out = widgets.Output(layout=center_align)
    prec_recall_out = widgets.Output()
    test_metrics_out = widgets.Output()





    # widgets
    rec_sel_dropdown_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='flex-start',
                    align_content='flex-start',
                    width='90%',
                    margin='0px 0px 20px 0px')

    rec_sel_label = widgets.Label(value="Select record by rank to view performance metrics",style=dict(font_weight='bold'))

    rec_sel_dropdown = widgets.Dropdown(
        options=['1','2','3','4','5','6','7','8','9','10'],
        value='1',
        description='',
        disabled=False
    )

    run_oos_button  = widgets.Button(description='Test model on out of sample data',layout=oos_test_layout,style=dict(
    button_color='darkred',
    text_color='white',
    font_weight='bold'
    ))



    # Containers

    record_selection_box = widgets.HBox([rec_sel_label,rec_sel_dropdown],layout=rec_sel_dropdown_layout)

    row_one_chart_box = widgets.HBox([record_out, roc_out],layout=top_row_layout)

    cm_box = widgets.HBox([cm_out,cm2_out],layout=cm_layout)

    prec_test_box = widgets.VBox([prec_recall_out],layout=prec_test_layout)

    row_two_chart_box = widgets.HBox([prec_test_box,cm_box])



    def initialize():
        tr.fit_model_with_record(0)

        with record_out:

            record_out.clear_output(wait=True)
            display(tr.print_records_to_table())

        with stats_out:
            stats_out.clear_output(wait=True)
            display(tr.show_train_metrics())

        with cm2_out:
            cm2_out.clear_output(wait=True)

        with cm_out:
            cm_out.clear_output(wait=True)
            display(tr.show_confusion_matrix_train())

        with roc_out:
            roc_out.clear_output(wait=True)
            display(tr.show_ROC())

        #with det_out:
         #   det_out.clear_output(wait=True)
          #  display(tr.show_DET())

        with prec_recall_out:
            prec_recall_out.clear_output(wait=True)
            display(tr.show_precision_recall())



    def select_record_to_view(dfx,names):
      
        val = int(names.new) - 1
        dfx.select_record(val)

        with stats_out:
            stats_out.clear_output(wait=True)
            display(dfx.show_train_metrics())

        with cm_out:
            cm_out.clear_output(wait=True)
            display(dfx.show_confusion_matrix_train())

        with roc_out:
            roc_out.clear_output(wait=True)
            display(dfx.show_ROC())

        #with det_out:
         #   det_out.clear_output(wait=True)
          #  display(dfx.show_DET())

        with prec_recall_out:
            prec_recall_out.clear_output(wait=True)
            display(dfx.show_precision_recall())




    def test_model_on_oos(dfx,val):

        dfx.set_testing_model()

        run_oos_button.description="Showing model performance"
        run_oos_button.disabled=True
        run_oos_button.style.button_color="black"

        with stats_out:
            stats_out.clear_output(wait=True)
            display(dfx.show_test_metrics())

        with cm_out:
            cm_out.clear_output(wait=True)
            display(dfx.show_confusion_matrix_train())
        
        with cm2_out:
            cm2_out.clear_output(wait=True)
            display(dfx.show_confusion_matrix_test())

        with roc_out:
            roc_out.clear_output(wait=True)
            display(dfx.show_ROC())

        #with det_out:
         #   det_out.clear_output(wait=True)
          #  display(dfx.show_DET())

        with prec_recall_out:
            prec_recall_out.clear_output(wait=True)
            display(dfx.show_precision_recall())

        with test_metrics_out:
            test_metrics_out.clear_output(wait=True)








    initialize()



    rec_sel_dropdown.observe(partial(select_record_to_view,tr),names='value')
    run_oos_button.on_click(partial(test_model_on_oos, tr))



    return widgets.VBox([record_selection_box,row_one_chart_box,row_two_chart_box,run_oos_button],layout=stat_box_layout)

In [55]:
tr = ResultsViewer(results,df)
eval_controller(tr)